# Introduction

Sometimes when working with CSV data, the users might want to convert each row into a custom type. This article discuss some aspect of this procedure. In the next sections, we will be working with the following custom type:

In [1]:
import csv
import io

class User:
    def __init__(self, uid, alias, shell):
        self.uid = int(uid)
        self.alias = alias
        self.shell = shell
    
    @classmethod
    def from_tuple(cls, tup):
        return cls(*tup)
    
    @classmethod
    def from_dict(cls, dic):
        return cls(**dic)
    
    def as_tuple(self):
        return (self.uid, self.alias, self.shell)
    
    def as_dict(self):
        return dict(uid=self.uid, alias=self.alias, shell=self.shell)
    
    def __repr__(self):
        return f'User(uid={self.uid!r}, alias={self.alias!r}, shell={self.shell!r})'

#
# CSV data sources
#
with_header = """
uid,alias,shell
501,karen,bash
502,john,tcsh""".strip().splitlines()

without_header = """
501,karen,bash
502,john,tcsh""".strip().splitlines()

# Map csv.reader to Custom Type

In [2]:
for row in map(User.from_tuple, csv.reader(without_header)):
    print(row)

User(uid=501, alias='karen', shell='bash')
User(uid=502, alias='john', shell='tcsh')


# Map csv.DictReader to Custom Type

In [3]:
for row in map(User.from_dict, csv.DictReader(with_header)):
    print(row)

User(uid=501, alias='karen', shell='bash')
User(uid=502, alias='john', shell='tcsh')


# Writing to csv.writer

In [4]:
user1 = User(501, 'karen', 'bash')
user2 = User(502, 'john', 'tcsh')

buffer = io.StringIO()
writer = csv.writer(buffer)

writer.writerow(user1.as_tuple())
writer.writerow(user2.as_tuple())

print(buffer.getvalue())

501,karen,bash
502,john,tcsh



# Writing to csv.DictWriter


In [5]:
user1 = User(501, 'karen', 'bash')
user2 = User(502, 'john', 'tcsh')

buffer = io.StringIO()
writer = csv.DictWriter(buffer, ['uid', 'alias', 'shell'])

writer.writerow(user1.as_dict())
writer.writerow(user2.as_dict())

print(buffer.getvalue())

501,karen,bash
502,john,tcsh



# Map csv.reader to a namedtuple

A `namedtuple` object works well with CSV data, plus, it is more lightweight than a class object. To convert from a tuple (which a CSV reader returns) to a `namedtuple` object, we use the `_make` method. The underscore in this case does not means `_make` is a private method, it is there to avoid field name collusion.

In [6]:
import collections
User2 = collections.namedtuple('User2', 'uid,alias,shell')

reader = csv.reader(without_header)
users = map(User2._make, reader)
for user in users:
    print(user)

User2(uid='501', alias='karen', shell='bash')
User2(uid='502', alias='john', shell='tcsh')


# Map csv.DictReader to namedtuple

Instead of using map as with previous case, we can use `itertools.starmap`:

In [7]:
import itertools

reader = csv.DictReader(with_header)
users = itertools.starmap(User2, reader)
for user in users:
    print(user)

User2(uid='uid', alias='alias', shell='shell')
User2(uid='uid', alias='alias', shell='shell')


# Write to csv.writer

Since `namedtuple` objects are tuples, they can be written directly to a CSV file without any modifications.

In [8]:
users = [
    User2(1001, 'alex', 'bash'),
    User2(1002, 'anna', 'tcsh'),
]

buffer = io.StringIO()
writer = csv.writer(buffer)
writer.writerows(users)

print(buffer.getvalue())

1001,alex,bash
1002,anna,tcsh



# Write to csv.DictWriter

In order to write a `namedtuple` object to a `csv.DictWriter`, we need to convert each object to a dictionary using the `._asDict` method. The `namedtuple` also has an attribute called `._fields` which can be handy for passing into the `csv.DictWriter` object:

In [9]:
# Convert tuples to dictionaries
users_as_dict = map(User2._asdict, users)

buffer = io.StringIO()
writer = csv.DictWriter(buffer, User2._fields)
writer.writerows(users_as_dict)

print(buffer.getvalue())

1001,alex,bash
1002,anna,tcsh



# Conclusion

If we need to, we can easily map rows of a CSV to and from custom types such as class objects or named tuple.